In [1]:
from losses import __get_target_idx
import datasets
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import Type
import torch
import pickle
os.chdir('/oscar/home/aloo1/thesis/humanlike_rule_learning_calibration')

/oscar/home/aloo1/interp-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('/users/aloo1/thesis/humanlike_rule_learning_calibration/datasets/kl_dataset/L1_hdists.pkl', 'rb') as f:
    p = pickle.load(f)
p

[tensor([[0.6250, 0.3750],
         [0.2083, 0.7917],
         [0.7500, 0.2500],
         [0.5000, 0.5000],
         [0.0833, 0.9167],
         [0.1667, 0.8333],
         [0.8750, 0.1250],
         [0.3333, 0.6667],
         [0.8333, 0.1667],
         [0.5833, 0.4167],
         [0.1304, 0.8696],
         [0.8696, 0.1304],
         [0.0435, 0.9565],
         [0.4783, 0.5217],
         [0.9130, 0.0870],
         [0.0870, 0.9130],
         [0.1739, 0.8261],
         [0.8261, 0.1739],
         [0.0000, 1.0000],
         [0.0435, 0.9565],
         [0.0435, 0.9565],
         [0.7826, 0.2174],
         [0.6957, 0.3043],
         [0.2174, 0.7826],
         [0.5652, 0.4348],
         [0.6957, 0.3043],
         [0.8261, 0.1739],
         [0.0870, 0.9130],
         [0.2609, 0.7391],
         [0.6522, 0.3478],
         [0.8696, 0.1304],
         [0.8261, 0.1739],
         [0.8261, 0.1739],
         [0.3043, 0.6957],
         [0.0870, 0.9130],
         [0.1304, 0.8696],
         [0.7826, 0.2174],
 

In [10]:
os.environ['TRANSFORMERS_CACHE'] = '/oscar/scratch/aloo1/model_cache_2'

tokenizer = AutoTokenizer.from_pretrained(
            'google/gemma-7b',
            cache_dir=os.environ['TRANSFORMERS_CACHE'], 
            add_eos_token=True)

In [11]:
data = datasets.load_from_disk("./datasets/kl_dataset")
train_dataset = data["L1"].filter(lambda x: x['concepts'] in ['hg02', 'hg04']).map(
            lambda e: tokenizer(e['text'], truncation=False, padding='max_length'), batched=True)

Map: 100%|██████████| 2/2 [00:00<00:00, 71.77 examples/s]


In [52]:

def get_target_idxes(
    tokenizer,
    input_ids: Type[torch.Tensor],
    return_padded_tensor: bool,
    for_causal_modeling: bool = True,
):
    """
    Get idxes of answer tokens for every tokenized paragraph in the dataset, so that
    answer tokens can be picked out of seq_len length tensor

    :param torch.Tensor input_ids: tokenized tensor
    :param bool return_padded_tensor: if True, will pad out all idxes with the last idx of each entry
    :param bool for_causal_modeling: if True, will minus 1 from the actual answer tokens because it is the previous word that predicts the next word
    """
    causal_offset = int(for_causal_modeling)
    target = [
        torch.Tensor(__get_target_idx(i, tokenizer)) - causal_offset for i in input_ids
    ]
    
    if return_padded_tensor:
        max_rows = max([len(row) for row in target])
        # print([len(row) for row in target])
        # padded = [torch.cat((x, torch.tile(x[-1], (max_rows-len(x), ))), axis=0) for x in target]
        padded = [x.reshape((1, -1)).expand(1,max_rows) for x in target]
        if len(padded) == 1:
            output = padded[0]
        else:
            output = torch.vstack(padded)
    else:
        output = target

    return output

In [53]:
get_target_idxes(tokenizer, train_dataset['input_ids'], True).shape

RuntimeError: The expanded size of the tensor (85) must match the existing size (75) at non-singleton dimension 1.  Target sizes: [1, 85].  Tensor sizes: [1, 75]